In [ ]:
#단, best_model.h5, test_stage_1, train_stage_1은 현재 실행하고 있는 폴더 내에 복사해서 붙여넣기 필수!
#층화 추출법
# DB받아야하고
# 개인별로 몇개할지 정하고
# 코드수정할 부분 파악하고
# 텍스트 파일이 2개씩 생성되니까 그거 파악해야하고
# 주말까진 해야하고
import re
import Sentimental_Analyzer as SA
import Date_List as DL
import BSC_Crawler as crawler
from Hot_Topic_Analyzer import hot_topic_analyzer
from Aspect_Miner import aspect_miner
import pandas as pd
import pymysql
import random
from selenium import webdriver
import csv

# 무작위 10일을 선택.
select_date = random.sample(range(1,30),15) # 1부터 29 중에 10개를 중복없이 뽑는다. -> type : list
print("선택 날짜 : ", select_date)


driver = webdriver.Chrome("C:/chromedriver.exe")

conn=pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='3721',
    db='webnoveldb',
    charset='utf8'
)

curs=conn.cursor()
sql="select * from novellink where (idnovel > 0 ) and (idnovel <1000)"
#개인 해당 범위에 대해서 novel_id 범위 수정할 것

curs.execute(sql)

rows=curs.fetchall()

novel_lists =[]
for row in rows:
    novel_lists.append(list(row)) #'작품ID', '작품제목', '조아라','문피아','카카오페이지','리디북스','시리즈온','네이버 웹소설']

crawler.ig_login(driver)
crawler.ridi_login(driver)
crawler.naver_login(driver)

for novel_list in novel_lists:
    
    #if(novel_list[0]<=324):
    #    continue
    
    #########################################################Text Rank#########################################################

    ####################인스타그램 크롤러################
    instagram_list=[]
    instagram_list=crawler.ig_scraping(driver,novel_list[1])

    ####################디시인사이드 크롤러##################
    dcinside_list=[]
    #dcinside_list=crawler.dc_scraping(driver, novel_list[1])

    ####################인스티즈 크롤러##############
    #instiz_list_list=[]
    #instiz_list=crawler.dc_scraping(driver, novel_list[985][1])


    ############크롤러 결과중 내용부분만 textfile화############
    f=open('Crawling\\{}.txt'.format(re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', novel_list[1])), 'a',encoding="utf-8")
    #with open('{}TR.txt'.format(title), 'a',encoding="utf-8") as f:
    for content in instagram_list:
        content[0]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[1])
        f.write("%s\n" % content[1])
    for content in dcinside_list:
        content[2]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[2])
        f.write("%s\n" % content[2])
        #for content in instiz_list:
        #    content[3]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[3])
        #    f.write("%s\n" % content[3])
    f.close()

    
    #########################################################Aspect Mining#########################################################

    ####################네이버 크롤러################
    naver_list=[]
    naver_list=crawler.naver_crawling(novel_list[1])

    ####################다음 크롤러##################
    daum_list=[]
    daum_list=crawler.daum_crawling(novel_list[1])

    ####################티스토리 크롤러##############
    tistory_list=[]
    tistory_list=crawler.tistory_crawling(novel_list[1])
    #[title, author, date, post]

    ############크롤러 결과중 내용부분만 textfile화############
    f=open('Crawling\\{}.txt'.format(re.sub('[-=+#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', novel_list[1])), 'a',encoding="utf-8")
    for content in naver_list:
        content[3]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[3])
        f.write("%s\n" % content[3])
    f.close()

    
    ############Textrank 분석############
    topic_list=[]
    try:
        topic_list=hot_topic_analyzer(novel_list[1])
        if(topic_list==None):
            topic_list=[]
            for i in range(0,5):
                topic_list.append('NULL')
    except:
        for i in range(0,5):
            topic_list.append('NULL')
            
    ###########Aspect mining 분석########\
    character_word_list=[]
    story_word_list=[]
    mood_word_list=[]
    try:
        character_word_list, story_word_list, mood_word_list = aspect_miner(novel_list[1])
        print(character_word_list)
        print(story_word_list)
        print(mood_word_list)
    except:
        for i in range(0,5):
            character_word_list.append('NULL')
            story_word_list.append('NULL')
            mood_word_list.append('NULL')

    print(topic_list)
    print(character_word_list)
    print(story_word_list)
    print(mood_word_list)
    
    f_w = open('TA_analysis_result.csv', 'a', encoding='cp949', newline='')
    wr = csv.writer(f_w)
    result=topic_list+character_word_list+story_word_list+mood_word_list
    result.insert(0,novel_list[0])
    wr.writerow(result)
    f_w.close()


    #########################################################감성분석#########################################################

    date_list=DL.make_date_list()

    if(novel_list[2]!=None):
        joara_comments=crawler.joara_comments(driver,novel_list[2])
        for review in joara_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1

    if(novel_list[5]!=None):
        ridi_comments=crawler.ridi_comments(select_date,driver,novel_list[5])
        for review in ridi_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1

    if(novel_list[6]!=None):
        serieson_comments=crawler.serieson_comments(select_date,driver,novel_list[7])
        for review in serieson_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1
    
    if(novel_list[7]!=None):
        naver_comments=crawler.naver_comments(driver,novel_list[7])
        for review in naver_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1
    


    f_w = open('S_analysis_result.csv', 'a', encoding='cp949', newline='')
    wr = csv.writer(f_w)
    for date in date_list:
        result=[]
        result.append(date[0])
        try:
            result.append("%.2f" % (date[1]/date[2]))
        except:
            result.append('0')
        result.insert(0,novel_list[0])
        wr.writerow(result)
    f_w.close()

선택 날짜 :  [29, 5, 9, 12, 27, 4, 15, 1, 17, 25, 14, 21, 24, 11, 16]
월영달을지키는그림자 검색 시작

총 0개의 데이터.

0 개의 데이터 받아오는 중.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
저장 완료.

월영  달을 지키는 그림자가 5주 후 유료화됩니다
연화
2014. 10. 18. 3:25
안녕하세요. 월영(月影) : 달을 지키는 그림자 한연화입니다. 한동안 차기작을 위해 취재도 하고 자료수집도 하고 그동안 못 읽었던 책도 마음껏 읽으면서 좀 쉬나 했더니 월영의 유료화가 결정되었네요. ㅠㅠㅠ 연재하는 동안 스트레스로 인해 없던 병까지 생길 지경이었는데(지금도 두통과 이명에 시달리고 있어요. 한연화를 살려주세요. ㅠㅠ) 다시 월영을 쓰기로 결정되었습니다. 이제 5주후면 월영은 5회까지만 무료로 공개되고, 나머지 회차는 모두 유료가 됩니다. 통권으로 정리해서 세 권쯤 나올 것 같습니다. 통권으로 정리할 때는 아쉬웠던 부분들도 많이 수정해서 넣을 생각입니다. 잘하면 이현의 이야기를 넣을 수 있을지도 모르겠네요.(넣는다면 리지가 이현에게 호의적인 시종들과 노예들을 모두 죽이는 장면 하며, 서하와 함께 있는데 이현이 밥상을 들고 왔다는 이유로 이현에게 상 들고 오라고 한 년 누구냐고 족쳐서 죽여버리는 장면, 이현이 서하의 방에 옷을 가져다 두러 갔다는 걸 알고 쫓아가서 죽기 직전까지 패는 장면, 다시 한 번 서하 눈에 띄면 그때는 두 눈과 혀를 뽑아버리고 길가에서 창기 노릇이나 하게 만들겠다고 협박하는 장면 등등 리지가 이현을 갈구는 장면이 주가 되겠네요.) 또 아신과 월영의 이야기도 조금 더 넣을 수 있을지 모르겠습니다. 내은이가 서하를 배신했던 사연에 대해서도요. 그러니 많이많이 빌려보시고, 사주세요. 이렇게 부탁드립니다. 굽신굽신(저 이제 실업자인데 취재 기간 1년 동안 생계는 유지해야 합니다. ㅠㅠㅠ)
월영  달을 지키는 그림자 구주 관련 설정 및 푸념
연화
2014. 11. 1. 2:44


19세기비망록 검색 시작

총 0개의 데이터.

0 개의 데이터 받아오는 중.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
저장 완료.


리뷰 1857) 19세기비망록 -조부경

늘보
2018. 6. 8. 16:39
	

저자
조부경


출판
파란미디어


발매
2013.09.02.

웹툰보기전에 원작소설 먼저 읽었어요.
릴리안 클리어워터5살에 입양된 릴리안병마에 지친 양아버지, 아버지에 지친 어머니두사람에게 지친 나.
피가 섞이지 않은 먼 사촌인줄 알았던 윌리엄 양아버지가 죽자 그가 친오라버니라며릴리안을 데리러 와요.
아름답고 평화로운 브루크사이드 대저택으로오게 된 릴리안.가져 온 짐속에서 발견한 오빠가 준 인형.인형속에서 나온 흑백가족사진
친오라비를 보며 설레어하는 지경.그를 이성으로 느끼는 나의 감정은 명백한 금기이다.이렇게 끌리게 된 남자가 하필이면 나의 오라버니라니...!여자의 비명소리.잠겨진 비밀의방감금된 여자일기장속의 흑백사진엘리엇이란 남자과연 누가 거짓말을 하고 있는 것일까?흑백사진과 일치하는 나의 꿈아니면 내가 알고 있는 현실의 윌 오라버니?
 
 
 
 
 
엘리엇 집으로 입양되어 온 윌리엄
파멸의 레온딘부부의아들.미친여자의 아들이라는 윌리엄이튼칼리지. 캠브리지대학도 함께가고윌리엄에 대한 소문
그날 밤. 윌리엄의 고백.그리고 호수가빠진 윌리엄.그리고 외면했던 엘리엇
모든사실을 들은 릴리안.
친부모님이 살았던 저택열어버린 비밀의방.마주해야했던 진실.그리고 엘리엇을 떠나기로 한 릴리안.양어머니와 시골로 내려가요.
프롤로그 푸른수염저택이야기에서부터결국 비밀의방을 열어버리고 진실을 알아버린 릴리안이야기.윌리엄과 엘리엇의비밀.1부느낌.2부느낌이 각기 다르게 흘러는미스테리하고 신비한 분위기가 건조한문체로쓰여진 19세기 비망록이었어요.뒷이야기가 계속 궁금했던 책이었어요.한가지 아쉬운건.에필이 너무나 짧았다는거.웹툰에서는 두사람이야기가 좀 더 나와주길요!
19세기 비망록  조부경  시대물 로맨스 
세잎클로버 나미
2013. 9. 15. 19:

19세기 비망록
라떼
2013. 9. 6. 10:45
19세기 비망록
리뷰보기
 
해가 지지 않는 나라란 말을 들은 영국 역사상 가장 전성기라고 일컬어지는 19세기 빅토리아 여왕 시대를 배경으로 한 로맨스 판타지 소설 '19세기 비망록'  이번에 알게 되었지만 인터넷 웹소설로 연재되면서 많은 독자들의 사랑을 받은 작품이라고 한다. 
 
조부경 작가님의 '19세기 비망록'은 프랑스 동화 작가 샤를 페로의 '푸른 수염'을 모티브로 만들어진 작품이라고 한다. 잔혹동화로 알려진 푸른 수염의 이야기는 대충 이러하다. 푸른 수염을 가진 남자가 있다. 그는 부자에 모든 것을 다 갖추고 있지만 그와 결혼한 여인들은 흔적도 없이 사라져 그를 두려워하는 사람들이 많다. 남자의 옆집에 사는 아름다운 여성들도 그의 청혼을 받게 된다.  처음에 두려움을 호감으로 이끌어 내어 결혼을 해서 새 아내를 얻은 푸른 수염...  그는 여행을 떠나기 전 아내에게 열쇠를 맡기며 다른 방은 다 되지만 지하의 작은 방에는 절대 문을 열지 말라고 한다. 사람이란 게 하지 말라고 하면 더 하고 싶은 심리가 있다. 보지 말라고 한 지하의 작은 방... 푸른 수염의 아내는 호기심을 이기지 못하고 방문을 열게 되고 이를 안 푸른 수염은 그녀를 죽이려고 한다. 죽음 직전 아내의 형제들의 도움으로 죽음에서 벗어나게 된다.
 
오늘도 악몽을 꾸고 깨어나는 '릴리안' 스물세 살의 아름다운 여성인 그녀는 생사의 기로에 서 있는 양아버지의 모습에 마음이 아프다. 거리감을 두고 대면대면하게 지내는 양어머니와 달리 사랑을 듬뿍 주었던 양아버지... 위독한 양아버지를 찾아 온 차가운 눈빛의 한 남자 레온딘 후작이 방문을 한다. 남자의 방문과 양아버지가 죽음... 장례식이 끝나고 릴리안의 오빠라고 말하는 레온딘 후작을 따라 브루크사이드의 대저택에 도착을 한다. 
 
이제 오빠를 의지하고 살아가야 한다. 자신과 전혀 닮지 않은 오빠 윌리엄을 믿고 따르는 마음과 달리 그를 대할 때마다 남모를 설렘이 생긴다. 오빠 윌리엄을 빼고 


19세기 비망록

쑥이만세
2018. 5. 2. 8:52
	
19세기 비망록 책이 웹툰으로 연재되고 있는거 아시나요?ㅎㅎ다음 웹툰이라 자주 보지는 못하는데.. ㅠㅠ초반에는 챙겨봤는데 한텀 쉬면서 그 뒤로는 못따라갔거든요.그런데 요래 출간하시네요~ ㅎㅎㅎ 요건 읽으면서 책 내용도 새록새록 기억하면서 읽고파서 질렀씁니다.
예전 리뷰가 있어서 찾아보니.. 초기에 썼던 리뷰라 그런지 내용은 많이 안쓰고 일상 생활 주저리만 써두었구만요..ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 
여튼 결국엔 출간 소식이 들리자 마자 질렀습니다!! 
예판 뜨는 날 사두기는 했는데 오늘 아침에 보니 이리 발송준비로 뜨고 있네요.1권먼저 받고 2권은.. 나중에 받는거루.. ㅎㅎㅎㅎ
작가님 블로그에서도 이벤트를 하니 혹시나 소장계획있으신 분들은 고고씽~~ https://blog.naver.com/syduam2452/221265699142

19세기 비망록  조부경
김권호
2013. 9. 8. 21:53
19세기 비망록
리뷰보기
애니 '페이트 스테이 나이트'는 마법사들이 역사속 인물들을 불러 싸움을 벌이는 내용입니다.
그중 기괴한 인물이 한명 나오는데...그의 이름은 '질 드레'입니다..
잔다르크의 전우였으며, 프랑스를 구한 귀족이자, 군인이지만...
전쟁후, 수많은 사람들을 고문하고 죽인 악명높은 연쇄살인마중 한명이기도 합니다..
 
그래서, '질 드레'를 모티브로 한 동화가 만들어졌고
그 동화가 바로 '푸른 수염'이지요
 
사실 이 책을 산 이유는, '푸른 수염'을 모티브로 한 이야기라고 해서 샀거든요
읽으신 이웃 누님이 '판타지 로맨스'라고 하셨는데
판타지적인 요소가 없는데요....마법이 나오는것도 아니고..
약간 호러틱하긴 한데....ㅋㅋㅋㅋ
초반부에는 읽다보니 얼마전에 읽었던 '레베카' 분위기도 나공...ㅋㅋㅋㅋㅋ
 
주인공 '릴리안'은 어려서 부모님을 잃고 '클린어워커가'에 입양이 됩니다.
그러나 그를 입양했던 양아버지가 죽자, 그녀는 갈곳이 없어집니다
그런 그녀에게 한 남자가 찾아옵니다
 
'

프로그램 종료합니다
Load...
Build...
Word2Vec Modeling finished
['순수하다', '힘들다', '흐뭇하다', '굉장하다', '훈훈하다']
['으스스하다', '비슷하다', '암울하다', '다르다', '싫다']
['넘치다', '훈훈하다', '순수하다', '같다', '발랄하다']
['릴리 안 ', '레온 딘 ', '윌리엄 ', '책 ', '자신 ']
['순수하다', '힘들다', '흐뭇하다', '굉장하다', '훈훈하다']
['으스스하다', '비슷하다', '암울하다', '다르다', '싫다']
['넘치다', '훈훈하다', '순수하다', '같다', '발랄하다']
절벽에피는꽃 검색 시작

총 0개의 데이터.

0 개의 데이터 받아오는 중.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
저장 완료.

절벽에 피는 꽃김지율발해
써보
2014. 3. 20. 23:17
절벽에 피는 꽃 상
리뷰보기
 
 출판사 소개글을 읽고 싶다면 드래그 ↓↓↓↓
김지율의 로맨스 소설. "잊으시옵소서. 소녀는 상감마마께 아무것도 아니었사옵니다." 여인의 이름은 비연이었다. 사람들은 그녀를 향해 절벽에 핀 꽃이라 하였다. 척박한 땅에 뿌리를 내려 온갖 궂은 비바람을 맞으면서도 모질게 견뎌 내는, 누구의 손에도 쉬이 꺾이지 않는 그녀를 그렇게 불렀다. "과인은… 나는 무슨 마음으로 지금을 살았겠소. 꿈에서라도 그대를 볼까, 내일 꿈에는 그대가 와 줄까, 그렇게 하루하루를 견뎌 왔소. 헌데 생시에서 만난 그대는 잊으라는 말만 하는구려. 차라리 꿈이라 하시오." 왕의 이름은 원주였다. 왕의 길에서 도망치려 하였으나 도망칠 수 없었고 왕이 되었기에 사랑하는 여인을 잃었으며, 그녀를 그리기 위해 왕으로 남았던 사내.
- 본문 중에서-
 
 
 
4.소분류: 시대물/황실로맨스/액션활극
 
5.평점:
 
 
 ★=1 ☆=0.5
5-1. 흡입력 : ★★★
5-2. 달달함: ★★★★
5-3. 코믹함: ★★★
5-4. 독특함: ★★ ☆
5-5. 캐릭터성:★★★ ☆
5-6. 스

무협 내 안에 마교있다초류희윤 126150화미  별335
바이라바
2019. 10. 12. 21:38
내 안에 마교있다 

리뷰보기






126~150화 내용 요약 (스포!) 

















임무발생.
세 척의 해적선이 접근.
아군 전력이 증원되지 않은 상황에서 적의 증원이 완료된 것.
기동타격대인 송유겸 일행이 와서 아군과 함께 싸우지만... 해적선에는 절정고수만 60여명.

초강자 다섯. 절정고수 20여명.
송유겸이 절정고수의 수를 최대한 빠르게 줄이고자 한다.
그래야 일행이 초강자를 상대하며 힘을 합칠 수 있을테니까.

송유겸은 악전고투를 이어나간다.
심장을 빗겨맞으며 하천에 떨어지기까지 하며...
빠르게 회회심공을 돌려 고통을 잠력, 내공화하며 이를 악무는 송유겸.

잠력을 내공으로 화하는 순간 천지의 기운까지 들어오는걸 느낀다.
그리고.. 절정에 올러선다.

기동타격조 또한, 실전을 겪으며 빠른 속도로 실력이 올라간다.

-----------------------------

일행이 있던 곳으로 이동해 송유겸 자신을 상처입힌 절정고수를 발견한다.
그를 유인해 제압하고, 고문해 정보를 얻는다.

상대가 내밭은 말 중, 중요한 단어들.
서장의 임지현, 복건 포전현의 동부 도서.

서장의 임지현은 사파 세력의 본거지.
복건 포전현의 동부 도서는 해적들의 지휘부.

사유 증운생. 사파의 구심점.
아무래도 모든 일의 원흉은 증운생인듯!

---------------------------------

송유하 병반으로 승반. 
첫째와 셋째도 한반씩 승반한다.

-------------------------------

송유겸이 이 정보를 제갈수광에게 알려주고, 제갈수광이 무림맹주 운천흠에게 전서구를 날려 알린다.
적드을 일망타진할 계획이었으나... 맹에서 맹주에 반하는 세력들에게 그 정보가 흘러들어가며... 난장판이 된다.
결국 적들이 본거지를 이동하기 전, 쳐야 하는 상황.

극비 임무(해적 본거지 소탕작전) 수행을 위해 복건으로 향한다.


Build...
Word2Vec Modeling finished
['송 유 ', '제갈 수광 ', '절정 고수 ', '동부 지맹 ', '천마 ']
['NULL', 'NULL', 'NULL', 'NULL', 'NULL']
['NULL', 'NULL', 'NULL', 'NULL', 'NULL']
['NULL', 'NULL', 'NULL', 'NULL', 'NULL']
총 댓글 수:  20
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
총 댓글 수:  8
[0, 0, 0, 1, 2, 1, 2, 0, 2, 0]
내머릿속에2000년마법역사 검색 시작

